## Importation of libaries

In [1]:
# Pytorch

import torch
import torch.nn as nn
import torchvision.models as models
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim
from torchvision import datasets, transforms
from torchvision.io import read_image
import torch.nn.functional as F


# Huggingface python image models

import timm

# Insightface models

from backbones import get_model

# Images Libs

from PIL import Image

# Numpy

import numpy as np

# Pandas

import pandas as pd


## Get config

In [2]:
from utils.utils_config import get_config

cfg = get_config('configs/ms1mv3_r50_onegpu')

## Initialize distributed

In [3]:
from torch import distributed

rank = 0
local_rank = 0
world_size = 1
distributed.init_process_group(
    backend="nccl",
    init_method="tcp://127.0.0.1:12584",
    rank=rank,
    world_size=world_size,
)

[W socket.cpp:464] [c10d] The server socket cannot be initialized on [::]:12584 (errno: 97 - Address family not supported by protocol).
[W socket.cpp:697] [c10d] The client socket cannot be initialized to connect to [::ffff:127.0.0.1]:12584 (errno: 97 - Address family not supported by protocol).


## CUDA



In [4]:
cuda_available = torch.cuda.is_available()
if cuda_available:
    print("CUDA is available!")
else:
    print("CUDA is not available. Switching to CPU mode.")
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

CUDA is available!


## Create Model

In [5]:
model = get_model('r50')
model.load_state_dict(torch.load('/root/face-rnd/dat/insightface/pytorch/r50ms1mv3.pth', map_location='cuda:0'))

<All keys matched successfully>

## Freeze All layers

In [6]:
for param in model.parameters():
    param.requires_grad = False  # Freeze all parameters


## Modify Final Layers

In [7]:
num_ftrs_in = model.fc.in_features  
# print(num_ftrs_in)
model.fc = nn.Linear(num_ftrs_in, 5013)  
model.features = nn.BatchNorm1d(5013, eps=1e-05)
nn.init.constant_(model.features.weight, 1.0)
model.features.weight.requires_grad = False

if cuda_available:
    model.to(device)

## Loss function and optimizer

In [ ]:
criterion = nn.CrossEntropyLoss().to(device)
# criterion = 

from losses import CombinedMarginLoss, ArcFace
from partial_fc_v2 import PartialFC_V2

# class AngularLoss(torch.nn.Module):
#     def __init__(self, radius=64.0, margin=5e-1):
#         super(AngularLoss, self).__init__()
#         self.radius = radius
#         self.cos_margin = torch.cos(torch.tensor(margin))
#         self.sin_margin = torch.sin(torch.tensor(margin))
#         self.cross_entropy = torch.nn.CrossEntropyLoss()

#     def forward(self, logits, labels):
#         target_mask = F.one_hot(labels, 5013)

#         sin_theta = torch.sqrt(1.0 - torch.pow(logits, 2))
#         arc_logits_target = logits * self.cos_margin - sin_theta * self.sin_margin
#         arc_logits = self.radius * ((1 - target_mask) * logits + target_mask * arc_logits_target)

#         loss = self.cross_entropy(arc_logits, labels)

#         return loss

class ArcFaceLoss(nn.Module):
    def __init__(self, num_classes, batch_size, s=30.0, m=0.5):
        super(ArcFaceLoss, self).__init__()
        self.num_classes = num_classes
        self.s = s
        self.m = m
        self.weight = nn.Parameter(torch.randn(num_classes, batch_size) * 0.01)

    def forward(self, logits, labels):
        logits = F.normalize(logits, dim=1)
        weight = F.normalize(self.weight, dim=1)

        # Compute the cos(theta)
        cos_theta = logits @ weight

        # Compute the cos(theta + m)
        cos_theta_m = cos_theta - self.m

        # Get one-hot encoded labels
        one_hot = torch.zeros_like(cos_theta)
        one_hot.scatter_(1, labels.view(-1, 1), 1)

        # Compute the final logits with margin
        logits_arc = cos_theta * (1 - one_hot) + cos_theta_m * one_hot
        logits_arc *= self.s

        # Compute the cross-entropy loss
        loss = F.cross_entropy(logits_arc, labels)

        return loss



criterion = ArcFaceLoss(5013, 384).to('cuda:0')


## Optimizer

In [9]:
# optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

optimizer = torch.optim.SGD(
    params=[{"params": criterion.parameters()}],
    lr=cfg.lr, momentum=0.9, weight_decay=cfg.weight_decay)

## Transform function

In [10]:
transform = transforms.Compose([
    transforms.Resize(112),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5677, 0.5188, 0.4885], std=[0.2040, 0.2908, 0.2848])
])

## Train Dataset loading

In [11]:
train_dataset = datasets.ImageFolder('/root/face-rnd/dat/personai_icartoonface_rectrain/icartoonface_rectrain', transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=384, shuffle=True)


In [ ]:
## Validation Set

class ImageDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.data = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        row = self.data.iloc[index]
        image_path = os.path.join(self.root_dir, row['filename'])
        image = read_image(image_path)
        label = row['classname']

        if self.transform:
            image = self.transform(image)

        return image, label


    
## Training The model

In [ ]:
for epoch in range(5):  # Adjust epochs as needed
    running_loss = 0.0
    for inputs, labels in train_loader[:1000]:
        # print(labels)
        inputs, labels = inputs.to(device), labels.to(device)
        # optimizer.zero_grad()
        outputs = model(inputs)
        # print(labels.shape)
        loss = criterion(outputs, labels)
        # print(loss)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        print(running_loss/len(inputs))
    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")


0.0575642337401708
0.1152564783891042
0.17289093633492789
0.2303431530793508
0.2877892901500066
0.3454662064711253
0.40277621646722156
0.46012458205223083
0.5177031457424164
0.5756306697924932
0.6331297556559244
0.6905472526947657
0.7478727002938589
0.8055095622936884
0.8631142377853394
0.9207677692174911
0.9779101312160492
1.0354635268449783
1.0930195649464924
1.1507463802893956
1.2082700282335281
1.2656527360280354
1.3231972654660542
1.380372663338979
1.4378630568583806
1.4954835424820583
1.5530586739381154
1.6104832192262013
1.6679461548725765
1.7253047426541646
1.7829482307036717
1.8407640606164932
1.8983220557371776
1.9552471488714218
2.0130089124043784
2.0710575729608536
2.128695776065191
2.1861060559749603
2.2437306692202887
2.300959289073944
2.3585280378659568
2.41621296107769
2.473773424824079
2.531663109858831
2.5888930410146713
2.6462378948926926
2.7035419742266336
2.7611120541890464
2.8183950384457908
2.8761296619971595
2.933626731236776
2.991241162021955
3.0484637320041656

In [ ]:
## Test some images

In [ ]:
model.eval()

def get_predicted_class(img_path):
    img_path = f'/root/face-rnd/dat/personai_icartoonface_rectrain/icartoonface_rectrain/{img_path}'
    x = model(transform(Image.open(img_path).convert('RGB')).to("cuda:0").unsqueeze(0))
    probabilities = nn.functional.softmax(x, dim=1)
    predicted_class = torch.argmax(probabilities, dim=1)
    return predicted_class.item()

In [ ]:
get_predicted_class('personai_icartoonface_rectrain_04096/personai_icartoonface_rectrain_04096_0000005.jpg')